In [1]:
import numpy as np
import math

In [3]:
def read_file(fileName):

    POS_TRACK_ID = 2
    POS_X = 4
    POS_Y = 5
    POS_T = 7

    X = []
    Y = []
    T = []
    TRACK_ID = []
    with open(fileName,'r') as file:
        lines = file.read().split("\n")
        terminos = lines[1].split(",") #con lines[1] me dirijo a la primera linea de datos
        trackId = int(terminos[POS_TRACK_ID]) #obtengo el primer trackId
        x = []
        y = []
        t = []
        i = 1
        while True:
            #print('hola')
            if lines[i]:
                terminos = lines[i].split(",")
                if trackId != int(terminos[POS_TRACK_ID]): #tengo un nuevo trackId
                    TRACK_ID.append(trackId)
                    trackId = int(terminos[POS_TRACK_ID])
                    #print(trackId)
                    X.append(x)
                    Y.append(y)
                    T.append(t)
                    x = []
                    y = []
                    t = []
                elif t == [] or float(terminos[POS_T]) != t[-1]:
                    x.append(float(terminos[POS_X]))
                    y.append(float(terminos[POS_Y]))
                    t.append(float(terminos[POS_T]))
                    i = i + 1
                else:
                    i = i + 1
            else:
                X.append(x)
                Y.append(y)
                T.append(t)
                TRACK_ID.append(trackId)
                break

    file.close()
    return X,Y,T,TRACK_ID

In [17]:
def avgPath(X,Y):
    
    NumberNewPoints = 5
    xPath = []
    yPath = []
    
    for i in range(len(X)-1):    
        xvals = np.linspace(X[i], X[i+1], NumberNewPoints+2) #crear 5 puntos en el medio (7 en total)
        yvals = np.linspace(Y[i], Y[i+1], NumberNewPoints+2)
        if i != 0:
            xvals = xvals[1:-1] #saco el primer y ultimo lugar
            yvals = yvals[1:-1]
        for j in range(len(xvals)):
            xPath.append(xvals[j]) #obtengo caminos con muchos puntos intermedios (5 entre cada dos puntos originales)    
            yPath.append(yvals[j])
    
    
    windowSize = NumberNewPoints*3     
    xPathSmooth = [] #se promedia con una ventana - al final se obtiene un camino con #windowSize puntos menos
    yPathSmooth = [] #la ventana movil se hace desde 0+windowsize/2 hasta fin-windowsize/2
    for i in range(math.trunc(windowSize/2),int(len(xPath)-math.ceil(windowSize/2))):
        xPathSmooth.append( np.mean (xPath [int(i-math.trunc(windowSize/2)):int(i+math.ceil(windowSize/2))] ) )
        yPathSmooth.append( np.mean (yPath [int(i-math.trunc(windowSize/2)):int(i+math.ceil(windowSize/2))] ) )    
    
    #uno el primer y ultimo punto con el avg path asi me queda una avg path completo

    xvals1 = np.linspace(X[0], xPathSmooth[0], math.ceil(windowSize/2)+2) #se completa la parte del smoothpath que
    yvals1 = np.linspace(Y[0], yPathSmooth[0], math.ceil(windowSize/2)+2) #falta entre el primer valor y el primero
    xvalsEnd = np.linspace(xPathSmooth[-1],X[-1], math.ceil(windowSize/2)+2) #del array smooth
    yvalsEnd = np.linspace(yPathSmooth[-1],Y[-1], math.ceil(windowSize/2)+2)    
    for i in range(len(xvals1)-1):
        xPathSmooth = [xvals1[-i-2]]+xPathSmooth+[xvalsEnd[i+1]]
        yPathSmooth = [yvals1[-i-2]]+yPathSmooth+[yvalsEnd[i+1]]

    return xPathSmooth, yPathSmooth


def VAP(X,Y,avgPathX,avgPathY,T):
    vel = []
    signo = []
    Index = []

    minIndexOld = 0        
    for j in range(1,len(X)):
        minDist = float('Inf')
        for i in range (len(avgPathX)):
            dist = math.sqrt((X[j]-avgPathX[i])**2+(Y[j]-avgPathY[i])**2)
            if dist < minDist:
                minIndex = i
                minDist = dist
        dist = 0
        Index.append(minIndex)
        if minIndex>=minIndexOld:
            signo.append(1)
            for i in range(minIndexOld,minIndex):
                dist = dist + math.sqrt((avgPathX[i+1]-avgPathX[i])**2+(avgPathY[i+1]-avgPathY[i])**2)
                
        else:
            signo.append(-1)
            for i in range(minIndex,minIndexOld):
                dist = dist - math.sqrt((avgPathX[i+1]-avgPathX[i])**2+(avgPathY[i+1]-avgPathY[i])**2) 
                
        minIndexOld = minIndex
        time = T[j]-T[j-1]
        vel.append(dist/time)
    vap = np.mean(vel)
    
    return vap,signo,vel,Index

In [18]:
filename = 'prueba_1track.csv'
X,Y,T,TRACK_ID = read_file(filename)

avgPathX,avgPathY = avgPath(X[0],Y[0])
vap,signo,vel,Index = VAP(X[0],Y[0],avgPathX,avgPathY,T[0])

print(X[0])
print(signo,vel,Index)

[0.0, 0.0, 0.0, 0.0, 1.386, 0.632, 0.0, 0.0]
[1, -1, 1, 1, 1, 1, 1] [0.7441601352763465, -0.36094902416523755, 0.515845225334259, 0.9904882197981978, 0.3640294693112756, 0.9490257275158738, 0.0] [13, 7, 15, 24, 27, 39, 39]
